In [3]:
import pandas as pd
import glob
import os


FOLDER = './BANG-A-Dataset'

file_csv = sorted(glob.glob(os.path.join(FOLDER, '*.csv')))
file_csv

['/Users/tuan/Desktop/house/cp-training/bao/BANG-A-Dataset/03_Customer_Behavior_Data_THPT.csv',
 '/Users/tuan/Desktop/house/cp-training/bao/BANG-A-Dataset/03_Item_information_Data_THPT.csv',
 '/Users/tuan/Desktop/house/cp-training/bao/BANG-A-Dataset/03_Shelf_Information_Data_THPT.csv']

In [6]:
data_behav = pd.read_csv(file_csv[0])
data_item = pd.read_csv(file_csv[1])
data_shelf = pd.read_csv(file_csv[2])

In [43]:
# 01. (4 points) List the 5 items that customers see the most.
# get the top 5 most popular items from  data_behav (Item ID, Shelf ID, count) and get the name of the items from data_item (Item ID, Shelf ID,)
top_5 = data_behav.groupby(['Item ID', 'Shelf ID']).size().reset_index(name='count').sort_values(by='count', ascending=False).head(5)
# get the value from data_item by ['Item ID', 'Shelf ID'] from top_5 and add the count column
top_5_v = data_item[data_item[['Item ID', 'Shelf ID']].apply(tuple, axis=1).isin(top_5[['Item ID', 'Shelf ID']].apply(tuple, axis=1))]
top_5_v = top_5_v.merge(top_5, on=['Item ID', 'Shelf ID'])
top_5_v

,Shelf ID,Item ID,Name,Origin,Price,Marketing strategy,Discount,Rate,count
0,0,4,Bánh trứng Custard,Thái Lan,"17,000 ₫",True,10,4,230
1,0,6,Bim bim Oishi,Vietnam,"12,000 ₫",False,5,5,233
2,0,7,Snack khoai tây Lays,Vietnam,"22,000 ₫",False,10,3,235
3,2,2,4 hộp sữa lúa mạch Milo 180ml,Vietnam,"30,000 ₫",False,0,5,233
4,2,8,Sữa ông thọ,Vietnam,"31,500 ₫",False,0,3,232


In [46]:
# 02. (7 points) List the 5 items that customers spend the most time looking at and holding.
# get the top 5 most popular items from  data_behav (Item ID, Shelf ID, count) and get the name of the items from data_item (Item ID, Shelf ID,)
# the time spent is the sum of the Looking at item (s) + Holding the item (s)
top_5 = data_behav.groupby(['Item ID', 'Shelf ID'])['Looking at item (s)', 'Holding the item (s)'].sum().sum(axis=1).reset_index(name='count').sort_values(by='count', ascending=False).head(5)
# get the value from data_item by ['Item ID', 'Shelf ID'] from top_5 and add the count column
top_5_v = data_item[data_item[['Item ID', 'Shelf ID']].apply(tuple, axis=1).isin(top_5[['Item ID', 'Shelf ID']].apply(tuple, axis=1))]
top_5_v = top_5_v.merge(top_5, on=['Item ID', 'Shelf ID'])
top_5_v

/var/folders/_3/y1q47dsx11165132ykf0nh640000gn/T/ipykernel_4118/2050186110.py:4: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  top_5 = data_behav.groupby(['Item ID', 'Shelf ID'])['Looking at item (s)', 'Holding the item (s)'].sum().sum(axis=1).reset_index(name='count').sort_values(by='count', ascending=False).head(5)


,Shelf ID,Item ID,Name,Origin,Price,Marketing strategy,Discount,Rate,count
0,0,6,Bim bim Oishi,Vietnam,"12,000 ₫",False,5,5,13866
1,0,7,Snack khoai tây Lays,Vietnam,"22,000 ₫",False,10,3,13362
2,2,8,Sữa ông thọ,Vietnam,"31,500 ₫",False,0,3,13939
3,7,6,Sữa chua uống Yakult,Vietnam,"26,000 ₫",False,0,3,22896
4,7,7,Sữa chua uống Probi,Vietnam,"36,000 ₫",False,5,5,22896


In [ ]:
# 03. (7 points) List the 5 most purchased items.
# data_behav[['Putting item into bag', 'Taking item out of bag', 'Putting item into bag in the 2nd time']].drop_duplicates().dropna()
# Putting item into bag, Taking item out of bag, Putting item into bag in the 2nd time
# purchase: [True, False, False], [True, True, True]
# if data is nan, then purchase = False

# Define the conditions for purchase
def check_purchase(row):
    # Check if any of the relevant columns have NaN values, return False if any are NaN
    if row.isnull().any():
        return False
    # Check if the condition for purchase is met
    if (row['Putting item into bag'] and not row['Taking item out of bag'] and not row['Putting item into bag in the 2nd time']) or \
       (row['Putting item into bag'] and row['Taking item out of bag'] and row['Putting item into bag in the 2nd time']):
        return True
    return False

# Apply the check_purchase function to each row of the DataFrame
data_behav['purchase'] = data_behav.apply(check_purchase, axis=1)

# 03. (7 points) List the 5 most purchased items. ((Item ID, Shelf ID, purchase = True) are considered as purchased items)
top_5_purchased_items = data_behav[data_behav['purchase'] == True].groupby(['Item ID', 'Shelf ID']).size().reset_index(name='purchase_count').sort_values(by='purchase_count', ascending=False).head(5)
# get the value from data_item by ['Item ID', 'Shelf ID'] from top_5 and add the count column
top_5_v = data_item[data_item[['Item ID', 'Shelf ID']].apply(tuple, axis=1).isin(top_5_purchased_items[['Item ID', 'Shelf ID']].apply(tuple, axis=1))]
top_5_v = top_5_v.merge(top_5_purchased_items, on=['Item ID', 'Shelf ID'])
top_5_v


In [66]:
# 04. (8 points) List the 5 items that are picked up and returned the most.
data_behav['pick_up_return'] = data_behav['Picking up item'] & data_behav['Returning item']
top_5 = data_behav.groupby(['Item ID', 'Shelf ID'])['pick_up_return'].sum().reset_index(name='count').sort_values(by='count', ascending=False).head(5)
# get the value from data_item by ['Item ID', 'Shelf ID'] from top_5 and add the count column
top_5_v = data_item[data_item[['Item ID', 'Shelf ID']].apply(tuple, axis=1).isin(top_5[['Item ID', 'Shelf ID']].apply(tuple, axis=1))]
top_5_v = top_5_v.merge(top_5, on=['Item ID', 'Shelf ID'])
top_5_v

,Shelf ID,Item ID,Name,Origin,Price,Marketing strategy,Discount,Rate,count
0,0,7,Snack khoai tây Lays,Vietnam,"22,000 ₫",False,10,3,127
1,2,2,4 hộp sữa lúa mạch Milo 180ml,Vietnam,"30,000 ₫",False,0,5,134
2,7,2,Sữa chua Vinamik,Vietnam,"25,400 ₫",False,0,3,114
3,7,12,Mý ý SG Food,Vietnam,"27,000 ₫",False,0,4,117
4,7,14,Nước lẩu Barona,Vietnam,"32,000 ₫",False,0,4,116


In [80]:
# 05. (8 points) Which customer groups by age (Teenagers: 18 - 30; Middle-aged: 31 - 60; Elderly: > 60) buy the most items?
# data_behav['Age'].unique(), data_behav['Age'].min(), data_behav['Age'].max()
# create the age group
data_behav['age_group'] = pd.cut(data_behav['Age'], bins=[0, 17, 30, 60, 100], labels=['Children', 'Teenagers', 'Middle-aged', 'Elderly'])
top_1 = data_behav.groupby('age_group')['purchase'].sum().reset_index(name='count').sort_values(by='count', ascending=False).head(1)
top_1

,age_group,count
2,Middle-aged,1583


In [82]:
data_behav

,Shelf ID,Person ID,Date,Time,Day,Age,Gender,Height,Weight,Married status,...,Holding the item (s),Holding the bag,Picking up item,Returning item,Putting item into bag,Taking item out of bag,Putting item into bag in the 2nd time,purchase,pick_up_return,age_group
0,4,1,2024-07-06,10:00:00,Saturday,52,Female,162,41,Married,...,36,True,True,False,False,False,True,False,False,Middle-aged
1,4,1,2024-07-06,10:02:00,Saturday,52,Female,162,41,Married,...,17,True,True,False,True,True,True,True,False,Middle-aged
2,4,1,2024-07-06,10:05:00,Saturday,52,Female,162,41,Married,...,39,True,True,True,True,True,False,False,True,Middle-aged
3,4,1,2024-07-06,10:06:00,Saturday,52,Female,162,41,Married,...,35,True,True,True,False,False,False,False,True,Middle-aged
4,4,1,2024-07-06,10:08:00,Saturday,52,Female,162,41,Married,...,16,True,True,False,False,False,True,False,False,Middle-aged
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15389,2,1500,2024-07-05,3:49:00,Friday,52,Female,154,44,Single,...,35,False,True,False,True,True,True,True,False,Middle-aged
15390,3,1500,2024-07-05,3:54:00,Friday,52,Female,154,44,Single,...,28,False,True,False,True,True,True,True,False,Middle-aged
15391,3,1500,2024-07-05,3:56:00,Friday,52,Female,154,44,Single,...,56,False,True,False,False,True,False,False,False,Middle-aged
15392,3,1500,2024-07-05,3:58:00,Friday,52,Female,154,44,Single,...,25,False,True,False,False,True,True,False,False,Middle-aged


In [92]:
# 06. (8 points) Which day of the week has the highest revenue?
# data_behav['Date'].unique()
# group by Person ID, Purchase, Date, and sum the Number of items * Price
# data_behav['revenue'] = data_behav['Number of items'] * data_item['Price']
# data_behav['day_of_week'] = pd.to_datetime(data_behav['Date']).dt.day_name()
# adding price to data_behav by merging with data_item on ['Item ID', 'Shelf ID']
data_behav = data_behav.merge(data_item[['Item ID', 'Shelf ID', 'Price']], on=['Item ID', 'Shelf ID'])
# convert price, for example 22,000 ₫ to 22000
data_behav['Price'] = data_behav['Price'].str.replace('₫', '').str.replace(',', '').astype(float)
# adding revenue to data_behav by checking the purchase, if yes then price, otherwise 0
data_behav['revenue'] = data_behav['purchase'] * data_behav['Price']
# data_behav
# group by 'Date' and sum the revenue
data_day =  data_behav.groupby('Date')['revenue'].sum().reset_index(name='count').sort_values(by='count', ascending=False)
# adding the Day of the week to the data_day from data_behav
data_day['Day'] = data_behav.groupby('Date')['Day'].first().reset_index()['Day']
data_day
# Saturday has the highest revenue


,Date,count,Day
5,2024-07-06,122195100.0,Saturday
6,2024-07-07,120424600.0,Sunday
3,2024-07-04,42260900.0,Thursday
1,2024-07-02,35841000.0,Tuesday
4,2024-07-05,32658400.0,Friday
2,2024-07-03,29629500.0,Wednesday
7,2024-07-08,16338000.0,Monday
0,2024-07-01,10759200.0,Monday


In [101]:
# 07. (7 points) Among the following 3 age groups: Teens (18 - 30), Middle-aged (31 - 60), Elderly: (> 60), which age group has the most people going to the supermarket?
# # group by 'age_group' and count distinct 'Person ID'
data_age = data_behav.groupby('age_group')['Person ID'].nunique().reset_index(name='count').sort_values(by='count', ascending=False)
data_age

,age_group,count
2,Middle-aged,753
3,Elderly,408
1,Teenagers,339
0,Children,0


In [123]:
# 08. (7 points) Top 5 discounted items that are most purchased by users?
# adding discount to data_behav by merging with data_item on ['Item ID', 'Shelf ID']
# if not discount then 0
data_behav = data_behav.merge(data_item[['Item ID', 'Shelf ID', 'Discount ']], on=['Item ID', 'Shelf ID'])
# select the purchase and discount > 0
data_behav['purchase_discount'] = data_behav['purchase'] & (data_behav['Discount '] > 0)
# group by 'Item ID', 'Shelf ID', and sum the purchase_discount
top_5 = data_behav.groupby(['Item ID', 'Shelf ID'])['purchase_discount'].sum().reset_index(name='count').sort_values(by='count', ascending=False).head(5)
# get the value from data_item by ['Item ID', 'Shelf ID'] from top_5 and add the count column
top_5_v = data_item[data_item[['Item ID', 'Shelf ID']].apply(tuple, axis=1).isin(top_5[['Item ID', 'Shelf ID']].apply(tuple, axis=1))]
top_5_v.columns = top_5_v.columns.str.strip()
top_5_v = top_5_v.merge(top_5, on=['Item ID', 'Shelf ID'])
top_5_v

,Shelf ID,Item ID,Name,Origin,Price,Marketing strategy,Discount,Rate,count
0,0,4,Bánh trứng Custard,Thái Lan,"17,000 ₫",True,10,4,78
1,1,10,Khăn tắm Shine,Vietnam,"115,000 ₫",True,20,5,47
2,2,7,Sữa bột Milo,Úc,"60,000 ₫",True,15,3,72
3,7,0,Kem tràng tiền,Vietnam,"45,700 ₫",True,5,5,71
4,7,7,Sữa chua uống Probi,Vietnam,"36,000 ₫",False,5,5,43


In [125]:
# 09. (7 points) Top 3 stalls with the most views?
# group by 'Shelf ID' and count distinct 'Person ID'
top_3 = data_behav.groupby('Shelf ID')['Person ID'].nunique().reset_index(name='count').sort_values(by='count', ascending=False).head(3)
# get the name of the stalls from data_shelf by 'Shelf ID'
top_3_v = data_shelf[data_shelf['Shelf ID'].isin(top_3['Shelf ID'])]
top_3_v = top_3_v.merge(top_3, on='Shelf ID')
top_3_v


,Shelf ID,Description,Width,Height,Length,Number of items,Shape,count
0,0,Quầy bánh kẹo,300,180,300,13,Quầy tròn,632
1,1,Quầy hoá mỹ phẩm,100,180,400,18,Kệ hàng,559
2,7,Quầy đông lạnh,700,180,100,16,Kệ hàng,718


In [128]:
# 10. (8 points) Among the most highly rated products, which are the top 3 most purchased products?
data_item['Rate'].unique()
# adding rate to data_behav by merging with data_item on ['Item ID', 'Shelf ID']
data_behav = data_behav.merge(data_item[['Item ID', 'Shelf ID', 'Rate']], on=['Item ID', 'Shelf ID'])
# select the purchase and rate > 4
data_behav['purchase_rate'] = data_behav['purchase'] & (data_behav['Rate'] > 4)
# group by 'Item ID', 'Shelf ID', and sum the purchase_rate
top_3 = data_behav.groupby(['Item ID', 'Shelf ID'])['purchase_rate'].sum().reset_index(name='count').sort_values(by='count', ascending=False).head(3)
# get the value from data_item by ['Item ID', 'Shelf ID'] from top_3 and add the count column
top_3_v = data_item[data_item[['Item ID', 'Shelf ID']].apply(tuple, axis=1).isin(top_3[['Item ID', 'Shelf ID']].apply(tuple, axis=1))]
top_3_v = top_3_v.merge(top_3, on=['Item ID', 'Shelf ID'])
top_3_v

,Shelf ID,Item ID,Name,Origin,Price,Marketing strategy,Discount,Rate,count
0,2,2,4 hộp sữa lúa mạch Milo 180ml,Vietnam,"30,000 ₫",False,0,5,47
1,7,0,Kem tràng tiền,Vietnam,"45,700 ₫",True,5,5,71
2,7,1,Kem vinamilk,Vietnam,"175,000 ₫",False,0,5,48


In [129]:
# 11. (4 points) Open-ended question: Find out the most popular items for specific customer groups (eg: Gender, marital status...). (Candidate's presentation should not exceed 500 words)
data_behav['Gender'].unique()

array(['Female', 'Male'], dtype=object)

In [132]:
# data_behav['Married status'].unique()
# group data by ['Gender', 'Married status', 'Item ID', 'Shelf ID'] and count the purchase
# Group by the specified columns and count the purchases (sum of True values)
purchase_count = data_behav.groupby(['Gender', 'Married status', 'Item ID', 'Shelf ID'])['purchase'].sum().reset_index()

# Rename the 'purchase' column to indicate it now contains the count of purchases
purchase_count.rename(columns={'purchase': 'purchase_count'}, inplace=True)

# Display the result
print(purchase_count)
# list the top 5 most purchased items for each group ('Gender', 'Married status')


     Gender Married status  Item ID  Shelf ID  purchase_count
0    Female        Married        0         0              10
1    Female        Married        0         1               5
2    Female        Married        0         2               4
3    Female        Married        0         3               4
4    Female        Married        0         4               5
..      ...            ...      ...       ...             ...
531    Male         Single       20         3               1
532    Male         Single       21         3               3
533    Male         Single       22         3               0
534    Male         Single       23         3               1
535    Male         Single       24         3               0

[536 rows x 5 columns]


In [138]:
# 11. (4 points) Open-ended question: Find out the most popular items for specific customer groups (eg: Gender, marital status...).
# Step 1: Group by 'Gender', 'Married status', 'Item ID', 'Shelf ID' and count purchases
purchase_count = data_behav.groupby(['Gender', 'Married status', 'Item ID', 'Shelf ID'])['purchase'].sum().reset_index()

# Rename the 'purchase' column to 'purchase_count'
purchase_count.rename(columns={'purchase': 'purchase_count'}, inplace=True)

# Step 2: Sort the purchases within each group of 'Gender' and 'Married status'
purchase_count = purchase_count.sort_values(by=['Gender', 'Married status', 'purchase_count'], ascending=[True, True, False])

# Step 3: Select the top 5 most purchased items for each 'Gender', 'Married status' group
top_5_per_group = purchase_count.groupby(['Gender', 'Married status']).head(5)

# Display the result
# print(top_5_per_group)
# adding the name of the items from data_item
top_5_v = data_item[data_item[['Item ID', 'Shelf ID']].apply(tuple, axis=1).isin(top_5_per_group[['Item ID', 'Shelf ID']].apply(tuple, axis=1))]
top_5_v = top_5_v.merge(top_5_per_group, on=['Item ID', 'Shelf ID'])
top_5_v
# sort top_5_v by [Gender Married, status, purchase_count] in ascending order
top_5_v = top_5_v.sort_values(by=['Gender', 'Married status', 'purchase_count'], ascending=[True, True, False])
# reordering the columns ['Gender', 'Married status', 'Item ID', 'Shelf ID', 'purchase_count', 'Item name']
top_5_v

,Shelf ID,Item ID,Name,Origin,Price,Marketing strategy,Discount,Rate,Gender,Married status,purchase_count
9,2,1,Lốc 4 hộp sữa tươi ít đường TH true MILK,Vietnam,"40,000 ₫",False,0,3,Female,Married,29
3,1,13,Tã dán Merries size NB 76 miếng (dưới 5kg),Nhật Bản,"385,000 ₫",False,0,3,Female,Married,28
7,1,14,Tã dán sơ sinh Bobby siêu mỏng Newborn 70 miế...,Nhật Bản,"215,000 ₫",False,0,3,Female,Married,27
17,7,0,Kem tràng tiền,Vietnam,"45,700 ₫",True,5,5,Female,Married,27
16,2,7,Sữa bột Milo,Úc,"60,000 ₫",True,15,3,Female,Married,26
4,1,13,Tã dán Merries size NB 76 miếng (dưới 5kg),Nhật Bản,"385,000 ₫",False,0,3,Female,Single,15
10,2,1,Lốc 4 hộp sữa tươi ít đường TH true MILK,Vietnam,"40,000 ₫",False,0,3,Female,Single,14
14,2,3,Sữa bột Ensure Gold lúa mạch,Hoa Kỳ,"905,000 ₫",False,0,3,Female,Single,14
15,2,4,Sửa bột Anele,New Zealand,"410,000 ₫",False,10,5,Female,Single,13
18,7,0,Kem tràng tiền,Vietnam,"45,700 ₫",True,5,5,Female,Single,13


In [ ]:
# 12. (10 points) Open-ended question: Are there any items that need to be rearranged in the store? # Why? (Candidate's presentation should not exceed 500 words)
Dựa vào thống kê từ câu hỏi 12, ta thấy các món hàng ở kệ 0, 1, 2, 7 được mua nhiều nhất, chủ cửa hàng cần xem xét việc sắp xếp lại các món hàng ở các kệ này để tăng hiệu quả bán hàng và đặt các kệ này ở vị trí thuận lợi nhất cho khách hàng.
Với mỗi kệ hàng dựa vào số lượng mua, chủ cửa hàng cần xem xét việc sắp xếp lại các món hàng trong cùng một kệ. Ví dụ: với kệ số 2: quầy sữa thì các loại sữa như: "Lốc 4 hộp sữa tươi ít đường TH true MILK", "Sữa bột Milo", "Sửa bột Anele" được mua nhiều nhất, chủ cửa hàng cần tăng số lượng hàng này và đặt ở vị trí thuận lợi cho khách hàng.
